<a href="https://colab.research.google.com/github/M-Yerro/IA2025/blob/main/24-IA2025%20MicroTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hola! Aquí hay un mini transformer para explorar. Es un fork del original de Andrej Karpathy.

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT. (https://www.youtube.com/watch?v=kCc8FmEb1nY)

In [1]:
# Vamos a usar un dataset... y no es MNIST!
!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

--2025-11-20 01:01:37--  https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1060259 (1.0M) [text/plain]
Saving to: ‘el_quijote.txt.2’

el_quijote.txt.2    100%[===================>]   1.01M  --.-KB/s    in 0.03s   

2025-11-20 01:01:37 (33.3 MB/s) - ‘el_quijote.txt.2’ saved [1060259/1060259]



In [2]:
# Lo abrimos
with open('el_quijote.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Largo del dataset en caracteres: ", len(text))

Largo del dataset en caracteres:  1038397


In [4]:
# Miramos los primeros mil
print(text[:1000])

DON QUIJOTE DE LA MANCHA
Miguel de Cervantes Saavedra

PRIMERA PARTE
CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha
En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años, era de complexión recia, sec

In [5]:
# Qué caracteres usa?
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-.0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijlmnopqrstuvxyz¡«»¿̀́̃̈–‘’“”
89


In [6]:
# Mapeamos los caracteres a enteros
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: entra string, salen integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: entran integers, sale string

print(encode("Buenas, como va?"))
print(decode(encode("Buenas, como va?")))

print(encode(chars))

[25, 71, 56, 64, 52, 69, 7, 1, 54, 65, 63, 65, 1, 72, 52, 23]
Buenas, como va?
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]


In [7]:
# codificamos todo el dataset a integers y lo metemos en un torch.Tensor
import torch # Sí, vamos a usar Pytorch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # Nuestro GPT verá los primeros mil caracteres así:

torch.Size([1038397]) torch.int64
tensor([27, 38, 37,  1, 40, 44, 32, 33, 38, 43, 28,  1, 27, 28,  1, 35, 24,  1,
        36, 24, 37, 26, 31, 24,  0, 36, 60, 58, 71, 56, 62,  1, 55, 56,  1, 26,
        56, 68, 72, 52, 64, 70, 56, 69,  1, 42, 52, 52, 72, 56, 55, 68, 52,  0,
         0, 39, 41, 32, 36, 28, 41, 24,  1, 39, 24, 41, 43, 28,  0, 26, 24, 39,
        32, 81, 43, 44, 35, 38,  1, 11, 20,  1, 40, 71, 56,  1, 70, 68, 52, 70,
        52,  1, 55, 56,  1, 62, 52,  1, 54, 65, 64, 55, 60, 54, 60, 65, 81, 64,
         1, 74,  1, 56, 61, 56, 68, 54, 60, 54, 60, 65,  1, 55, 56, 62,  1, 57,
        52, 63, 65, 69, 65,  1, 59, 60, 55, 52, 62, 58, 65,  1, 27,  9,  1, 40,
        71, 60, 61, 65, 70, 56,  1, 55, 56,  1, 62, 52,  1, 36, 52, 64, 54, 59,
        52,  0, 28, 64,  1, 71, 64,  1, 62, 71, 58, 52, 68,  1, 55, 56,  1, 62,
        52,  1, 36, 52, 64, 54, 59, 52,  7,  1, 55, 56,  1, 54, 71, 74, 65,  1,
        64, 65, 63, 53, 68, 56,  1, 64, 65,  1, 67, 71, 60, 56, 68, 65,  1, 52,
      

In [8]:
# Separamos los datos en train y validation
n = int(0.9*len(data)) # El primer 90% es train, el resto val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([27, 38, 37,  1, 40, 44, 32, 33, 38])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Cuando entra {context} el target es: {decode([target.item()])}")

Cuando entra tensor([27]) el target es: O
Cuando entra tensor([27, 38]) el target es: N
Cuando entra tensor([27, 38, 37]) el target es:  
Cuando entra tensor([27, 38, 37,  1]) el target es: Q
Cuando entra tensor([27, 38, 37,  1, 40]) el target es: U
Cuando entra tensor([27, 38, 37,  1, 40, 44]) el target es: I
Cuando entra tensor([27, 38, 37,  1, 40, 44, 32]) el target es: J
Cuando entra tensor([27, 38, 37,  1, 40, 44, 32, 33]) el target es: O


In [11]:
torch.manual_seed(1337)
batch_size = 4 # Cuántas secuencias en paralelo?
block_size = 8 # Cuál es el largo máximo para predecir?

def get_batch(split):
    # Preparamos un batch pequeño de datos: inputs x --- targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Cuando entra {context.tolist()} sale: {target}")

inputs:
torch.Size([4, 8])
tensor([[70, 52, 55,  1, 55, 56,  1, 66],
        [ 7,  1, 54, 65, 64,  1, 56, 62],
        [60,  1, 52,  1, 63, 60, 81,  1],
        [54, 52, 68, 52,  7,  1, 64, 60]])
targets:
torch.Size([4, 8])
tensor([[52, 55,  1, 55, 56,  1, 66, 52],
        [ 1, 54, 65, 64,  1, 56, 62,  1],
        [ 1, 52,  1, 63, 60, 81,  1, 63],
        [52, 68, 52,  7,  1, 64, 60,  1]])
----
Cuando entra [70] sale: 52
Cuando entra [70, 52] sale: 55
Cuando entra [70, 52, 55] sale: 1
Cuando entra [70, 52, 55, 1] sale: 55
Cuando entra [70, 52, 55, 1, 55] sale: 56
Cuando entra [70, 52, 55, 1, 55, 56] sale: 1
Cuando entra [70, 52, 55, 1, 55, 56, 1] sale: 66
Cuando entra [70, 52, 55, 1, 55, 56, 1, 66] sale: 52
Cuando entra [7] sale: 1
Cuando entra [7, 1] sale: 54
Cuando entra [7, 1, 54] sale: 65
Cuando entra [7, 1, 54, 65] sale: 64
Cuando entra [7, 1, 54, 65, 64] sale: 1
Cuando entra [7, 1, 54, 65, 64, 1] sale: 56
Cuando entra [7, 1, 54, 65, 64, 1, 56] sale: 62
Cuando entra [7, 1, 54, 65,

In [12]:
print(xb) # La entrada de nuestro transformer

tensor([[70, 52, 55,  1, 55, 56,  1, 66],
        [ 7,  1, 54, 65, 64,  1, 56, 62],
        [60,  1, 52,  1, 63, 60, 81,  1],
        [54, 52, 68, 52,  7,  1, 64, 60]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Cada token lee directamente los logits del siguiente desde esta tabla
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx y targets son ambos (B,T) tensores de enteros
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx es (B, T) array de indices en el contexto actual
        for _ in range(max_new_tokens):
            # obtener las predictions
            logits, loss = self(idx)
            # foco en el último paso
            logits = logits[:, -1, :] # se vuelve (B, C)
            # applicar softmax
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample desde la distribucion
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append index sampleado a la secuencia en ejecución
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 89])
tensor(5.2697, grad_fn=<NllLossBackward0>)

vWUAF]”0lUz;eW0[xo'1IXOV(iO7LyE.rstT(d“l;8Xa3b4H7"d̈:,dulhXS6.M2(rz¡̈¡9GfN!(Kf5¿Q6BRBdOqg3,AO]D
N’L–


In [14]:
# instanciamos un optimizador
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(100): # más pasos, mejores resultados

    # sampleamos un batch
    xb, yb = get_batch('train')

    # evaluamos el error
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.858426094055176


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


8<dS??<o9i‘8v-iiNip–itQ;4.Q'́"e!X2Ds5YXHY5q̀5ed̈dXHYERgub¿G[NJP‘yM9¿31–“.]a“¡«7!yy[̈
yGK!–”!K–"T‘?['–u)KOs(Lh»aHy RgCG6iZD;:BXTre6iKzJ–3mOKj0],4Nz
caH]41tVE.Jp̈:('̈b¿6 Bn:Lt-“¡̃'–jy¡!1Ru
6tW.6«l'oN:7FOEV́vf6<a¿»8«41g,Sl1
[a”̈:l4PG9o?non<SgLq<XLvLHYzU–“:3ORCst-nfg<́”)Id<́uiI,¡ziLn Be¡MV'GPvWTM4H̃DM’Aufg8"bn8"d1‘YaoOr0a:RMVMQ8eW!B̃7źOD¿B‘̈¡I3(¡h
̀HmWiMpLed̈7¿;
dVV'’m
Eb]¿QLEV Z'41́,S?“‘(P‘fa“Y5u«x“Fc1U)Vyl0s(8vpBũg'7J!lfD?C<EM9yMpP0J(zGnd’́M h“TDn'-DNe)Bxb‘O.0L?x?PVT(rE4N:RM;pzKRb¿q3mno3h8MVrOvu


In [17]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=600)[0].tolist()))

# Sí, yo creo que anda joya... pero no lo entrenamos, no?


H 3O.H5‘PUW6H́l4:F?”zSfrD́la“¡.)uM1JTCS?x2’7EćRd’́R5zyv,SYsKdr:p¿¡<RRhc<oN‘Cd5-
; MP;QL?,̀WU8CF<bs(r.u:.Q<.‘“3PĨP;CEV?6'"9–1GL'h7f0–ZpNJ2qd4uBRmQ̈W4–]a“e(TJ"ë¿:ODB.‘l3”2‘;[-¡XCHc‘5–EcuM'́ECG;emOUzaIYld6́6́‘qGWlFgX:zXTd“uQ l',Sinzt]K“c
rSgz0m̃A»B-6aGpl37RCpe m»3(yE”[̈7LLd»jBJ3bO?–N:–:‘ai‘GLv6–Ejfm :“'–? q.l'́N-67u<!¡“”c1̀Kd.mRlz-5IOK–JT’́ljH’8r[ć;]<̀PXT<D¿5btzKco'Ax–3qcaW̃(àq‘?Tf(uxZ¿Q2S»"QYa!692,SluaI<(H.6‘"<?[f<dv)s6C9,)p¿Y«Gj?-:]dOzVsO
ibYzbp:5DH]q‘“:eyEV“Y’VI)y!,̈-<TCMCj!nF<‘¡SD5̈h0<RB9Jp7<bv́<0qLcaNlx2̈f]caT?xb3cY;:5̈8ClD»–Vm3̈?[<j!??lz‘P;Oz)A;O
9WiMpo?¡̃;fW2p–LO gYWmEu’(̃ J36“eBBZ"Ta


## Self-attention

Notes:
- La atención es un mecanismo de comunicación. Puede entenderse como nodos en un grafo dirigido, que se apuntan mutuamente y van agregando informacio´n ponderada con una suma pesada de todos los nodos que lo apuntan.
- El modelo no tiene noción de espacio. Hay atención sobre un conjunto de vectroes. Necesitamos codificar los tokens posicionalmente.
- Cada dimensión se procesa independientemente.
- Si es un bloque encoder, se saca el enmascaramiento `tril`, así todos los tokens se comunican. Un decoder tiene enmascaramiento triangular, y ese usa en contextos autorregresivos (modelado de lenguaje).
- "self-attention" significa que las keys y values se producen igual que las queries. "cross-attention": las queries se producen desde x, pero las keys y values vienen de otra fuente (módulo encoder).
- Si es "Scaled" attention, se divide `wei` por 1/sqrt(head_size). Esto hace que cuando la entrada Q,K respeta unit variance, wei también la respetará, y la softmax seguirá "difusa" y no saturará (mucho).

### MicroGPT:



In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 9000
#max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

with open('el_quijote.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


--2025-11-20 01:01:43--  https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1060259 (1.0M) [text/plain]
Saving to: ‘el_quijote.txt.3’

el_quijote.txt.3    100%[===================>]   1.01M  --.-KB/s    in 0.03s   

2025-11-20 01:01:43 (31.5 MB/s) - ‘el_quijote.txt.3’ saved [1060259/1060259]

0.212825 M parameters
step 0: train loss 4.6837, val loss 4.6888
step 100: train loss 2.3703, val loss 2.4025
step 200: train loss 2.2507, val loss 2.2982
step 300: train loss 2.1992, val loss 2.2474
step 400: train loss 2.1578, val loss 2.1908
step 500: train loss 2.1007, val loss 2.1226
step 600: train loss 2.0657, val loss 2.1029
st

In [19]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


-1quien que él diced que su ni respondía Dorotea. ¡Aho? Hecha podía, y por agusaré, cercala en la provechada en grado MachaOY cómo era de sacas, aunque yo soy bajados de que acaban, y de me buenos demagase echaramentes reduplia. Yo esto, le hace aragoltados que acertosa haciendos perosable salio, que no tomar en el cuestrada, a desdad quiso de tiern razo se puedez que tomarrados colices de Amohecharse hallaba vio presisa. Porque ibajababa desta pala.
Y coyo? sosobre de caldenga, huéndad de más desamos -oso quitar muchos entrovechos su exto, el y ni haclaría en limpo) a saber. Todóse lo que declario de la bocadecnta y este en el otro, puesto me sobre haceder conga, lo que otra avio, resió hubierse le vuestrido si no fles, entenderó y él dado entrecho y porque no las más acobre con gran los la ventideros de la vecebida, crestamente los de Triste con él, -respresto que su señores y del pocor de margos han de habían poco podrán, yo te quiego, los dos yelmas el verdade duc